In [1]:
from model.dataloader import NinaProDatasetLoader, NearlabDatasetLoader
from model.nina_helper import *
import torch
import pandas as pd
from sklearn.utils import shuffle
from scipy import signal
import os
import matplotlib.pyplot as plt
from model.utils import list_files, take_n_shots, take_n_samples_from_every_class
from model.model import CNet2D

import warnings

warnings.filterwarnings("ignore")

In [2]:
path_nearlab = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/data/nearlab/8features/person"
path_ninapro = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/data/ninapro"
path_nearlab_combo = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/data/nearlab/combo/person"
results_basic = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/basicEval"
results_ninapro = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/ninapro_basic"
results_nearlab = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/nearlab"
results_prototype_num = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/prototype_num"
results_prototype_num2 = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/prototype_num2"
results_fsl = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/few-shot"
results_fsl_ninapro = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/few-shot-ninapro"
results_subject = "E:/Dennis_Bachelor/Bachelor_Thesis-main/src/results/subject"
versions = ["GLVQ", "Softmax", "GMLVQ"]
prototype_version = ["GLVQ", "GMLVQ"]
n_shots = [2, 5, 10]
prototypes_per_class = [1,4,8,16]

# Nearlab
---

# Load in all Subject Data

In [3]:
# Runs all models with all subjects expect one as training and one as testing
def run_model_all_subjects():
    for person in range(1, 12):
        test_paths = list_files(path_nearlab + f"{person}", "csv")
        train_paths = []
        for person2 in range(1, 12):
            if person2 != person:
                train_files = list_files(path_nearlab + f"{person2}", "csv")
                train_paths.extend(train_files)
        
        data = NearlabDatasetLoader(train_paths, test_paths)
        X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()

        for version in versions:
            # Recommended Settings
            current_model = CNet2D(version=version, epochs=25, batch_size=128, learning_rate=0.0002)
            history = current_model.fit(X_train, y_train, 8, X_val, y_val)
            # Add accuracy of model to history
            evaluation = current_model.evaluate_model(X_test, y_test)
            history["accuracy"] = str(evaluation["accuracy"])
            # Checks if folder exists, if not creates it
            if not os.path.exists(results_subject + f"/person{person}/"):
                os.makedirs(results_subject + f"/person{person}/")
            # Save model
            current_model.save_model_state(results_subject + f"/person{person}/model_{version}_Subject_{person}")
            # Save history
            current_model.save_history_csv(history, results_subject + f"/person{person}/history_{version}_Subject_{person}")


# Num Prototypes

In [4]:
# Runs all models with all hand-orientation combinations on each subject where 2 files are used for training and 1 for testing
def run_model_all_orientations_num_prototypes(prototypes_per_class):
    for person in range(1, 4):
        all_files = list_files(path_nearlab + f"{person}", "csv")
        data = NearlabDatasetLoader(all_files[:2], all_files[2:])
        X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()
        for version in prototype_version:
            for num_prototype_per_class in prototypes_per_class:
                # Recommended Settings
                current_model = CNet2D(version=version, epochs=250, batch_size=128, learning_rate=0.0002, num_prototypes_per_class=num_prototype_per_class)
                history = current_model.fit(X_train, y_train, 20, X_val, y_val)
                # Add accuracy of model to history
                evaluation = current_model.evaluate_model(X_test, y_test)
                history["accuracy"] = str(evaluation["accuracy"])
                # Checks if folder exists, if not creates it
                if not os.path.exists(results_prototype_num2 + f"/person{person}/"):
                    os.makedirs(results_prototype_num2 + f"/person{person}/")
                # Save model
                current_model.save_model_state(results_prototype_num2 + f"/person{person}/model_{version}_Prototypes_{num_prototype_per_class}")
                # Save history
                current_model.save_history_csv(history, results_prototype_num2 + f"/person{person}/history_{version}_Prototypes_{num_prototype_per_class}")

## All hand orientations

In [5]:
# Runs all models with all hand-orientation combinations on each subject where 2 files are used for training and 1 for testing
def run_model_all_orientations():
    for person in range(8, 12):
        for orien in range(0, 3):
            all_files = list_files(path_nearlab + f"{person}", "csv")
            test_paths = [all_files[orien]]
            train_paths = [x for x in all_files if x not in test_paths]
            data = NearlabDatasetLoader(train_paths, test_paths)
            X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()

            for version in versions:
                # Recommended Settings
                current_model = CNet2D(version=version, epochs=150, batch_size=128, learning_rate=0.0002)
                history = current_model.fit(X_train, y_train, 10, X_val, y_val)
                # Add accuracy of model to history
                evaluation = current_model.evaluate_model(X_test, y_test)
                history["accuracy"] = str(evaluation["accuracy"])
                # Checks if folder exists, if not creates it
                if not os.path.exists(results_basic + f"/person{person}/"):
                    os.makedirs(results_basic + f"/person{person}/")
                # Save model
                current_model.save_model_state(results_basic + f"/person{person}/model_{version}_Orientation_{orien}")
                # Save history
                current_model.save_history_csv(history, results_basic + f"/person{person}/history_{version}_Orientation_{orien}")

## Few-shot

In [6]:
def run_k_shot_FSL(k_shots):
    for person in range(6, 12):
        train_files = list_files(path_nearlab + f"{person}", "csv")
        test_files = list_files(path_nearlab_combo + f"{person}", "csv")
        data = NearlabDatasetLoader(train_files, test_files)
        X_train, y_train, X_val, y_val, X_test, y_test = data.split_few_shot_learning()
        y_test -= 1
        for version in prototype_version:
            if os.path.exists(results_fsl_ninapro + f"/person{person}/model_{version}"):
                model = CNet2D(version=version, epochs=5, batch_size=128, learning_rate=0.0002)
                model.load_model_state(results_fsl + f"/person{person}/model_{version}/model_state.pth")
                history = {"accuracy": []}
            else:
                model = CNet2D(version=version, epochs=5, batch_size=128, learning_rate=0.0002)
                history = model.fit(X_train, y_train, 10, X_val, y_val)
                if not os.path.exists(results_fsl + f"/person{person}/"):
                    os.makedirs(results_fsl + f"/person{person}/")
                model.save_model_state(results_fsl_ninapro + f"/person{person}/model_{version}")
            model.save_model_state(results_fsl + f"/person{person}/model_{version}")
            for k_shot in k_shots:
                avg_acc = []
                for _ in range(5):
                    # Check if clone is doing what its supposed to do
                    curr_model = model.clone()
                    for c in torch.unique(y_test):
                        X_random, y_random = take_n_shots(X_test, y_test, k_shot, c.item())
                        curr_model.add_new_class(X_random)
                        curr_model.optimize_new_prototypes(X_random, epochs=40, batch_size=16)
                    eval = curr_model.evaluate_model(X_test, y_test)
                    avg_acc.append(eval["accuracy"])
                history["accuracy"] = str(avg_acc)
                model.save_history_csv(history, results_fsl + f"/person{person}/model_{version}_K_Shot_{k_shot}.csv")
                    

# Datasets
---

In [69]:
person=4
full_path = path_ninapro + "/" + f"DB2_s{person}"
data = NinaProDatasetLoader(full_path, person, 2)
X_train_nina, y_train_nina, X_val_nina, y_val_nina, X_test_nina, y_test_nina = data.load_few_shot_learning_data()

In [73]:
version = "GMLVQ"

In [ ]:
current_model = CNet2D(version=version, num_classes=40, epochs=50, batch_size=128, learning_rate=0.0002, dataset_type="NinaPro")
history = current_model.fit(X_train_nina, y_train_nina, 10, X_val_nina, y_val_nina)

if not os.path.exists(results_fsl_ninapro + f"/person{person}/"):
    os.makedirs(results_fsl_ninapro + f"/person{person}/")
current_model.save_model_state(results_fsl_ninapro + f"/person{person}/model_{version}")

for k_shot in [1,2,5,10,20,40]:
    avg_acc = []
    for _ in range(5):
        # Check if clone is doing what its supposed to do
        curr_model = current_model.clone()
        for c in torch.unique(y_test_nina):
            X_random, y_random = take_n_shots(X_test_nina, y_test_nina, k_shot, c.item())
            print(X_random.shape)
            curr_model.add_new_class(X_random)
            curr_model.optimize_new_prototypes(X_random, epochs=40)
        eval = curr_model.evaluate_model(X_test_nina, y_test_nina)
        avg_acc.append(eval["accuracy"])
    history["accuracy"] = str(avg_acc)
    current_model.save_history_csv(history, results_fsl_ninapro + f"/person{person}/model_{version}_K_Shot_{k_shot}.csv")

# Ninapro
---

In [7]:
def run_standard_ninapro():
    for person in range(8, 9):
        full_path = path_ninapro + "/" + f"DB2_s{person}"
        data = NinaProDatasetLoader(full_path, person, 2)
        X_train, y_train, X_val, y_val, X_test, y_test = data.load_data()
        number_classes = len(torch.unique(y_train))
        versions = ["GMLVQ"]
        for version in versions:
            # Recommended Settings
            current_model = CNet2D(version=version, num_classes=number_classes, epochs=200, batch_size=128, learning_rate=0.0002, dataset_type="NinaPro")
            history = current_model.fit(X_train, y_train, 20, X_val, y_val)
            # Add accuracy of model to history
            evaluation = current_model.evaluate_model(X_test, y_test)
            history["accuracy"] = str(evaluation["accuracy"])
            # Checks if folder exists, if not creates it
            if not os.path.exists(results_ninapro + f"/person{person}/"):
                os.makedirs(results_ninapro + f"/person{person}/")
            # Save model
            current_model.save_model_state(results_ninapro + f"/person{person}/model_{version}")
            # Save history
            current_model.save_history_csv(history, results_ninapro + f"/person{person}/history_{version}")

In [33]:
def run_k_shot_FSL_ninapro(k_shots):
    for person in range(1, 12):
        full_path = path_ninapro + "/" + f"DB2_s{person}"
        data = NinaProDatasetLoader(full_path, person, 2)
        X_train_nina, y_train_nina, X_val_nina, y_val_nina, X_test_nina, y_test_nina = data.load_few_shot_learning_data()
        """
        if os.path.exists(results_fsl_ninapro + f"/person{person}/model_{version}"):
            model = CNet2D(version=version, num_classes=number_classes, epochs=5, batch_size=128, learning_rate=0.0002, dataset_type="NinaPro")
            model.load_model_state(results_fsl_ninapro + f"/person{person}/model_{version}")
            history = {"accuracy": []}
        else:
        """
        model = CNet2D(version="GLVQ", num_classes=40, epochs=5, batch_size=128, learning_rate=0.0002, dataset_type="NinaPro")
        history = model.fit(X_train_nina, y_train_nina, 20, X_val_nina, y_val_nina)
        if not os.path.exists(results_fsl_ninapro + f"/person{person}/"):
            os.makedirs(results_fsl_ninapro + f"/person{person}/")
        model.save_model_state(results_fsl_ninapro + f"/person{person}/model_GLVQ")
        
        for k_shot in k_shots:
            avg_acc = []
            for _ in range(5):
                # Check if clone is doing what its supposed to do
                curr_model = model.clone()
                for c in torch.unique(y_test_nina):
                    X_random, y_random = take_n_shots(X_test_nina, y_test_nina, k_shot, c.item())
                    curr_model.add_new_class(X_random)
                    curr_model.optimize_new_prototypes(X_random, epochs=40)
                eval = curr_model.evaluate_model(X_test_nina, y_test_nina)
                avg_acc.append(eval["accuracy"])
            history["accuracy"] = str(avg_acc)
            model.save_history_csv(history, results_fsl_ninapro + f"/person{person}/model_GLVQ_K_Shot_{k_shot}.csv")